<a href="https://colab.research.google.com/github/Gracepicharporn/Data-Science-project/blob/main/classification/SVD_Classification_appilance_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Differnt method and applied SVD

## UK-DALE

### Building1

Dowload Dataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Image with fridge**

In [4]:
dir = '/content/gdrive/MyDrive/data-device/SVD-fridge'

train_datagen = ImageDataGenerator(
      samplewise_std_normalization= True,
      validation_split = 0.2)

train_generator =train_datagen.flow_from_directory(
     dir,  # train folder
     batch_size = 64,  # define batch size
     shuffle=True,  # shuffle the train dataset
     subset = 'training',
     target_size = (224,224)

)

test_generator =train_datagen.flow_from_directory(
     dir,  # test folder
     batch_size= 1,  # define batch size (1 for test dataset)
     shuffle=True,  
     subset = 'validation',
     target_size = (224, 224)

)
                    

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 2608 images belonging to 2 classes.
Found 650 images belonging to 2 classes.


In [5]:
# shape of dataset
trainShape=train_generator.__getitem__(0)[0].shape  # train shape
testShape=test_generator.__getitem__(0)[0].shape  # test shape
print("Train Shape \n",trainShape)
print("Test Shape \n",testShape)

Train Shape 
 (64, 224, 224, 3)
Test Shape 
 (1, 224, 224, 3)


In [6]:
from tensorflow.keras.applications import VGG16  # VGG16 pre-trained model
import tensorflow as tf  # for deep learning
from tensorflow.keras import Model  # for creating a model
from sklearn.metrics import confusion_matrix, classification_report  # for calculating precision, recall, f1-score, and accuracy
import pandas as pd  # for creating a table
import numpy as np  # for calculating numeric values
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping  # for learning rate reduction and early stopping
lr_reduce = ReduceLROnPlateau(monitor='accuracy', patience=1)  # learning rate reduction
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)  # early stopping

def pretrained_model(model):
  '''
    Create a model from a pre-trained model by adding 1 3-nodes dense layer on top
    of the model
    Input:
      model: a pre-trained model
    Output:
      a model 
  '''
  # define a pre-trained model without the top layers, and with the default weights
  pre_trained_model = model(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
  pre_trained_model.trainable = False  # freeze the weights
  pre_trained_model.summary()

  x = tf.keras.layers.Flatten()(pre_trained_model.output)  # flatten layer
  x = tf.keras.layers.Dense(2 , activation='sigmoid')(x)  # 1 3-nodes dense layer
  model = Model(pre_trained_model.input, x) 
  model.summary()

  return model

def train(model):
  '''
    train the model
    Input:
      model: a model
    Output:
      history of the training process
  '''
  # compile the model with specific configuration
  model.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])
  # train the model for 10 epochs and 30 steps for each epoch
  history = model.fit(train_generator, validation_data = test_generator, epochs = 30, 
                      callbacks=[lr_reduce, early_stop])
  return history

def plot_train_evaluation(history, model):
  '''
    Plot train loss and acuracy. Then, evaluate the model (precision, recall, f1-score, and accuracy) with test dataset
    Input:
      history: a history of the training process
      model: a model
    Output:
      test report (DataFrame)
  '''
  # train loss and accuracy plot
  plt.figure(figsize=(12, 8))  # define figure size
  plt.subplot(2, 2, 1)  # create a subplot
  plt.plot(history.history['val_accuracy'], label='val_accuracy') 
  plt.plot(history.history['accuracy'], label='train_accuracy') 
  plt.legend()  # plot legend
  plt.xlabel('epochs')  # define x label
  plt.title('Accuracy plot')  # define title

  plt.subplot(2, 2, 2)  # create a subplot
  plt.plot(history.history['val_loss'], label='val_loss') 
  plt.plot(history.history['loss'], label='train_loss') 
  plt.legend()  # plot legend
  plt.xlabel('epochs')  # define x label
  plt.title('Loss plot')  # define title
  plt.show()  # show all plots

  # print out average, max, and min of train accuracy and loss
  print('average train accuracy: ', sum(history.history['accuracy'])/len(history.history['accuracy']))
  print('max train accuracy: ', max(history.history['accuracy']))
  print('average train loss: ', sum(history.history['loss'])/len(history.history['loss']))
  print('min train loss: ', min(history.history['loss']))

  print('#########################################################')
  # print out average, max, and min of validation accuracy and loss
  print('average train accuracy: ', sum(history.history['val_accuracy'])/len(history.history['val_accuracy']))
  print('max train accuracy: ', max(history.history['val_accuracy']))
  print('average train loss: ', sum(history.history['val_loss'])/len(history.history['val_loss']))
  print('min train loss: ', min(history.history['val_loss']))


In [7]:
# VGG16 pre-trained model
model1 = pretrained_model(VGG16)

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# train the model
history1 = train(model1)

Epoch 1/30
 8/41 [====>.........................] - ETA: 5:02 - loss: 0.7999 - accuracy: 0.5273

In [ ]:
# training and testing results
plot_train_evaluation(history1, model1)

In [ ]:
model1.save("svd_model.h5")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16  
import tensorflow as tf 
from tensorflow.keras import Model  
from sklearn.metrics import confusion_matrix, classification_report  

In [ ]:
model2 = load_model('svd_model.h5')

In [ ]:
model2.summary()

**Kettle**

In [ ]:
dir2 = '/content/gdrive/MyDrive/data-device/SVD-kettle'
train_datagen2 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator2 = train_datagen2.flow_from_directory(
     dir2,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict1 = model2.evaluate(train_generator2, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict1 

In [ ]:
model3 = load_model('svd_model.h5')

**Washer Dryer**

In [ ]:
dir3 = '/content/gdrive/MyDrive/data-device/SVD-washer'
train_datagen3 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator3 = train_datagen3.flow_from_directory(
     dir3,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict2 = model3.evaluate(train_generator3, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict2

### Across building

**fridge**

In [ ]:
dir4 = '/content/gdrive/MyDrive/data-device/UK-DALE-data/diff/SVD-across-house/train-fridge-SVD'
train_datagen4 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator4 = train_datagen4.flow_from_directory(
     dir4,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict4 = model3.evaluate(train_generator4, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict4

**Kettle**

In [ ]:
dir5 = '/content/gdrive/MyDrive/data-device/UK-DALE-data/diff/SVD-across-house/train-kettle-SVD'
train_datagen5 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator5 = train_datagen5.flow_from_directory(
     dir5,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict5 = model3.evaluate(train_generator5, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict5

**Washer Dryer**

In [ ]:
dir6 = '/content/gdrive/MyDrive/data-device/UK-DALE-data/diff/SVD-across-house/train-washer-SVD'
train_datagen6 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator6 = train_datagen6.flow_from_directory(
     dir6,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict6 = model3.evaluate(train_generator6, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict6

## REDD

### Building 1

**fridge**

In [ ]:
dir7 = '/content/gdrive/MyDrive/data-device/REDD-data/diff/buildind1-SVD/train-fridge-svd'
train_datagen7 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator7 = train_datagen7.flow_from_directory(
     dir7,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict7 = model3.evaluate(train_generator7, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict7 

**Microwave**

In [ ]:
dir8 = '/content/gdrive/MyDrive/data-device/REDD-data/diff/buildind1-SVD/train-microwave-svd'
train_datagen8 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator8 = train_datagen8.flow_from_directory(
     dir8,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict8 = model3.evaluate(train_generator8, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict8 

**Washer Dryer**

In [ ]:
dir9 = '/content/gdrive/MyDrive/data-device/REDD-data/diff/buildind1-SVD/train-washer-svd'
train_datagen9 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator9 = train_datagen9.flow_from_directory(
     dir9,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict9 = model3.evaluate(train_generator9, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict9 

### Across builindg

**Fridge**

In [ ]:
dir10 = '/content/gdrive/MyDrive/data-device/REDD-data/diff/across-builg-SVD/SVD-train_fridge'
train_datagen10 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator10 = train_datagen10.flow_from_directory(
     dir10,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict10 = model3.evaluate(train_generator10, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict10

**Microwave**

In [ ]:
dir11 = '/content/gdrive/MyDrive/data-device/REDD-data/diff/across-builg-SVD/SVD-train_microwave'
train_datagen11 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator11 = train_datagen11.flow_from_directory(
     dir11,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict11 = model3.evaluate(train_generator11, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict11

**Washer Dryer**

In [ ]:
dir12 = '/content/gdrive/MyDrive/data-device/REDD-data/diff/across-builg-SVD/SVD-train_washer'
train_datagen12 = ImageDataGenerator(
      samplewise_std_normalization= True,
      )

train_generator12 = train_datagen12.flow_from_directory(
     dir12,  # train folder
     shuffle=True,  # shuffle the train dataset
     target_size = (224,224)

)

In [ ]:
y_predict12 = model3.evaluate(train_generator12, return_dict = True, use_multiprocessing= True)

In [ ]:
y_predict12